In [ ]:
#import cac thu vieb can thiet
from tensorflow import keras
from tensorflow.keras import layers
import os
import numpy as np
import cv2
from glob import glob
from sklearn import model_selection
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger, TensorBoard
from tensorflow.keras.metrics import Precision, Recall, MeanIoU
import matplotlib.pyplot as plt
from tensorflow.keras.utils import CustomObjectScope
from tqdm import tqdm
from tensorflow.keras.optimizers import Adam, Nadam, SGD

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/'My Drive'/xla

/content/drive/My Drive/xla


In [ ]:
#parameter
size_unet=256
#Lua chon cac sieu tham so Hyperparameters
batch=8
lr = 1e-4
epochs = 35

In [ ]:
#Ham tra ve tap train, tap valid va tap test
def load_data():
    #Tach tap du lieu thanh 80% cho train, 10% cho val, 10% cho test
    split=0.1;
    images = sorted(glob( "images/*"))
    masks = sorted(glob("masks/*"))
    #glob giup hoan thien cac duong dan

    total_size = len(images)
    valid_size = int(split * total_size)
    test_size = int(split * total_size)
    #random_state tham so cho viec tao so ngau nhien va 42 la so thuong duoc dung
    train_x, valid_x = model_selection.train_test_split(images, test_size=valid_size, random_state=42)
    train_y, valid_y = model_selection.train_test_split(masks, test_size=valid_size, random_state=42)

    train_x, test_x = model_selection.train_test_split(train_x, test_size=test_size, random_state=42)
    train_y, test_y = model_selection.train_test_split(train_y, test_size=test_size, random_state=42)
    #print("Tong so anh {}".format(total_size))
    #print("So anh trainning {}".format(len(train_y)))
    #print("So anh dung de danh gia {}".format(len(valid_x)))
    #print("So anh dung de test {}".format(len(test_x)))
    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)

In [ ]:
load_data()

In [ ]:
#Ap dung voi tap val va tap test
#Doc hinh anh sau do chuan hoa
def read_image(path):
    path=path.decode()
    x = cv2.imread(path,cv2.IMREAD_COLOR)
    #Voi dau vao la mot hinh anh mau
    x = cv2.resize(x, (size_unet,size_unet))
    x = x/255.0
    return x

def read_mask(path):
    path=path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, (size_unet, size_unet))
    x = x/255.0
    #Them cho du 3 chieu
    x = np.expand_dims ( x, axis = -1)
    return x
#Ham chuyen cac anh ve dang tensorflow
def tf_convert(x, y):
    def _convert(x, y):
      x=read_image(x)
      y=read_mask(y)
      return x,y
    #Chuyen sang dang tensor de dua vao mo hinh
    x, y = tf.numpy_function(_convert, [x, y], [tf.float64, tf.float64])
    #Xac nhan lai hinh dang
    x.set_shape([size_unet, size_unet, 3])
    y.set_shape([size_unet, size_unet, 1])
    return x, y
#Chuan bi de du lieu hoat dong hieu qua voi GPU
#Neu khong co viec chuan bi nay GPU se phai doi CPU load xong du lieu gay ton kem thoi gian
#Cach giai quyet la xu ly song song (pipeline) khi GPU xu ly thi CPU tien hanh chuan bi 
#Dieu nay thuc hien bang cach su dung Tensorflow Dataset API

def tf_dataset(x, y, batch=8):
    #Ham from_tensor_slices() để tạo 1 Dataset objects
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    #Sua lai dung dinh dang ta can thong qua ham tf_convert
    dataset = dataset.map(tf_convert)
    #Chia du lieu theo batch da dinh truoc
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    #Train batch qua nhieu epoch
    dataset = dataset.repeat()
    return dataset


In [ ]:
#Tang cuong du lieu va chuan hoa hinh anh voi cac lop cua Keras
#cac seed tao so ngau nhien phai giong nhau de phu hop giua anh goc va mat na
def Augment(seed=37):
    img = tf.keras.Input(shape=(None,None,3))
    msk = tf.keras.Input(shape=(None,None,1))

    #Lat hinh anh ca tren va duoi
    i = tf.keras.layers.RandomFlip("horizontal",seed=seed)(img)
    m = tf.keras.layers.RandomFlip("horizontal",seed=seed)(msk)
    #Dich vi tri hinh anh mot cach ngau nhien
    i = tf.keras.layers.RandomTranslation((-0.15, 0.15),(-0.15, 0.15),seed=seed)(i)
    m = tf.keras.layers.RandomTranslation((-0.15, 0.15),(-0.15, 0.15),seed=seed)(m)
    #Xoay hinh anh
    i = tf.keras.layers.RandomRotation((-0.15, 0.15),seed=seed)(i)
    m = tf.keras.layers.RandomRotation((-0.15, 0.15),seed=seed)(m)
    #Zoom hinh anh
    i = tf.keras.layers.RandomZoom((-0.1, 0.05),(-0.1, 0.05),seed=seed)(i)
    m = tf.keras.layers.RandomZoom((-0.1, 0.05),(-0.1, 0.05),seed=seed)(m)
    #Chuan hoa lai hinh anh
    i = tf.keras.layers.Rescaling(scale=1./255)(i)
    m = tf.keras.layers.Rescaling(scale=1./255)(m)
    
    return tf.keras.Model(inputs=(img,msk), outputs=(i,m))
Augment = Augment()

In [ ]:
#Ap dung voi tap train
#Doc hinh anh sau do chuan hoa
def read_image_train(path):
    path=path.decode()
    x = cv2.imread(path,cv2.IMREAD_COLOR)
    #Voi dau vao la mot hinh anh mau
    x = cv2.resize(x, (size_unet,size_unet))
    x = x/1.0
    return x

def read_mask_train(path):
    path=path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, (size_unet, size_unet))
    x = x/1.0
    #Them cho du 3 chieu
    x = np.expand_dims ( x, axis = -1)
    return x
#Ham chuyen cac anh ve dang tensorflow
def tf_convert_train(x, y):
    def _convert_train(x, y):
      x=read_image_train(x)
      y=read_mask_train(y)
      return x,y
    #Chuyen sang dang tensor de dua vao mo hinh
    x, y = tf.numpy_function(_convert_train, [x, y], [tf.float64, tf.float64])
    #Xac nhan lai hinh dang
    x.set_shape([size_unet, size_unet, 3])
    y.set_shape([size_unet, size_unet, 1])
    return x, y
#Chuan bi de du lieu hoat dong hieu qua voi GPU
#Neu khong co viec chuan bi nay GPU se phai doi CPU load xong du lieu gay ton kem thoi gian
#Cach giai quyet la xu ly song song (pipeline) khi GPU xu ly thi CPU tien hanh chuan bi 
#Dieu nay thuc hien bang cach su dung Tensorflow Dataset API
def tf_dataset_train(x, y, batch=8):
    #Ham from_tensor_slices() để tạo 1 Dataset objects
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    #Sua lai dung dinh dang ta can thong qua ham tf_convert_train
    dataset = dataset.map(tf_convert_train)
    #Chia du lieu theo batch da dinh truoc
    dataset = dataset.batch(batch).map(lambda x,y: Augment((x,y)))
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    #Train batch qua nhieu epoch
    dataset = dataset.repeat()
    return dataset


In [ ]:
#Ham danh gia dice coef
smooth = 1.
def dice_coef(y_true, y_pred):
    y_true_f = tf.keras.layers.Flatten()(y_true)
    y_pred_f = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth)


def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

In [ ]:
(train_x,train_y), (valid_x,valid_y),(test_x,test_y)=load_data()

train_dataset = tf_dataset_train(train_x, train_y, batch=batch)
valid_dataset = tf_dataset(valid_x, valid_y, batch=batch)
test_dataset = tf_dataset(test_x, test_y, batch=batch)

In [ ]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
def conv_block(x, num_filters):
    x = Conv2D(num_filters, (3, 3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, (3, 3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def build_model():
    size = 256
    num_filters = [64, 128, 256, 512, 1024]
    inputs = keras.Input((size_unet, size_unet, 3))

    skip_x = []
    x = inputs
    ## Encoder
    for f in num_filters:
        x = conv_block(x, f)
        skip_x.append(x)
        x = MaxPool2D((2, 2))(x)

    ## Bridge
    x = conv_block(x, num_filters[-1])

    num_filters.reverse()
    skip_x.reverse()
    ## Decoder
    for i, f in enumerate(num_filters):
        x = UpSampling2D((2, 2))(x)
        xs = skip_x[i]
        x = Concatenate()([x, xs])
        x = conv_block(x, f)

    ## Output
    x = Conv2D(1, (1, 1), padding="same")(x)
    x = Activation("sigmoid")(x)

    return Model(inputs, x)

In [ ]:
model = build_model()
opt = tf.keras.optimizers.Adam(lr)
metrics = ["acc",tf.keras.metrics.Recall(), tf.keras.metrics.Precision(), dice_coef, MeanIoU(num_classes=2)]
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=metrics)

In [ ]:
#Cac tham so ta se quan sat
callbacks = [
          #Luu bo trong so weight    
          ModelCheckpoint("files/model.h5"),
          #Theo doi loss function va giam LR
          #factor gia tri learning rate se giam 
          #patience: sau so chu ky ma LR se giam
          ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3),
          #Truyen lai ket qua vao CSV
          CSVLogger("files/data.csv"),
          TensorBoard(),
          #Theo doi loss function
          #patience: sau so chu ky neu khong cai thien thi dung
          #restore_best_weights: lua chon khoi phuc tu epoch co ket qua tot nhat
          EarlyStopping(monitor='val_loss', patience=12, restore_best_weights=True)
     ]

In [ ]:
train_steps = len(train_x)//batch
valid_steps = len(valid_x)//batch


if len(train_x) % batch != 0:
        train_steps += 1
if len(valid_x) % batch != 0:
        valid_steps += 1

history=model.fit(train_dataset,
validation_data=valid_dataset,
epochs=epochs,
steps_per_epoch=train_steps,
validation_steps=valid_steps,
callbacks=callbacks,
)

In [ ]:
print(history.history.keys())
# summarize history for dice_coef
plt.plot(history.history['dice_coef'])
plt.plot(history.history['val_dice_coef'])
plt.title('Ket qua dice_coef')
plt.ylabel('dice_coef')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for accuracy
plt.plot(history.history['mean_io_u'])
plt.plot(history.history['val_mean_io_u'])
plt.title('Ket qua mean_io_u')
plt.ylabel('mean_io_u')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Ket qua loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
test_steps = (len(test_x)//batch)
if len(test_x) % batch != 0:
  test_steps += 1
with CustomObjectScope({'dice_loss': dice_loss, 'dice_coef': dice_coef}):
  model = tf.keras.models.load_model("files/model.h5")
model.evaluate(test_dataset, steps=test_steps)

13/13 [==============================] - 28s 2s/step - loss: 0.1617 - acc: 0.9389 - recall: 0.7440 - precision: 0.9106 - dice_coef: 0.7692 - mean_io_u: 0.4244


[0.161725252866745,
 0.9388812184333801,
 0.7439764142036438,
 0.9105837345123291,
 0.7692307829856873,
 0.4244335889816284]

In [ ]:
def read_image(path):
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (256, 256))
    x = x/255.0
    return x

def read_mask(path):
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, (256, 256))
    x = np.expand_dims(x, axis=-1)
    return x

def mask_parse(mask):
    mask = np.squeeze(mask)
    mask = [mask, mask, mask]
    mask = np.transpose(mask, (1, 2, 0))
    return mask

In [ ]:
for i, (x, y) in tqdm(enumerate(zip(test_x, test_y)), total=len(test_x)):
        x = read_image(x)
        y = read_mask(y)
        y_pred = model.predict(np.expand_dims(x, axis=0))[0] > 0.5
        h, w, _ = x.shape
        white_line = np.ones((h, 10, 3)) * 255.0

        all_images = [
            x * 255.0, white_line,
            mask_parse(y), white_line,
            mask_parse(y_pred) * 255.0
        ]
        image = np.concatenate(all_images, axis=1)
        cv2.imwrite(f"results/{i}.png", image)

100%|██████████| 100/100 [01:18<00:00,  1.28it/s]
